## Deliverable 2. Create a Customer Travel Destinations Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [27]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Eureka,US,40.8021,-124.1637,51.01,83,1,7.00,clear sky
1,1,Bembereke,BJ,10.2283,2.6634,71.71,25,4,3.71,clear sky
2,2,Mataura,NZ,-46.1927,168.8643,81.88,46,13,1.63,few clouds
3,3,Cape Town,ZA,-33.9258,18.4232,70.00,83,20,11.50,few clouds
4,4,New Norfolk,AU,-42.7826,147.0587,66.00,47,39,3.00,scattered clouds


In [28]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 80


In [29]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Eureka,US,40.8021,-124.1637,51.01,83,1,7.00,clear sky
1,1,Bembereke,BJ,10.2283,2.6634,71.71,25,4,3.71,clear sky
3,3,Cape Town,ZA,-33.9258,18.4232,70.00,83,20,11.50,few clouds
4,4,New Norfolk,AU,-42.7826,147.0587,66.00,47,39,3.00,scattered clouds
5,5,Kapaa,US,22.0752,-159.3190,73.40,49,40,6.91,scattered clouds
6,6,Gat,IL,31.6100,34.7642,55.00,100,0,1.70,clear sky
7,7,Yanam,IN,16.7333,82.2167,68.00,88,86,9.22,mist
9,9,Touros,BR,-5.1989,-35.4608,77.00,83,20,3.44,few clouds
10,10,Saint George,US,37.1041,-113.5841,57.20,17,1,11.50,clear sky
11,11,Busselton,AU,-33.6500,115.3333,62.01,95,100,3.00,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
clean_df= preferred_cities_df.count()
clean_df

City_ID                350
City                   350
Country                343
Lat                    350
Lng                    350
Max Temp               350
Humidity               350
Cloudiness             350
Wind Speed             350
Current Description    350
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Eureka,US,51.01,clear sky,40.8021,-124.1637,
1,Bembereke,BJ,71.71,clear sky,10.2283,2.6634,
3,Cape Town,ZA,70.00,few clouds,-33.9258,18.4232,
4,New Norfolk,AU,66.00,scattered clouds,-42.7826,147.0587,
5,Kapaa,US,73.40,scattered clouds,22.0752,-159.3190,
6,Gat,IL,55.00,clear sky,31.6100,34.7642,
7,Yanam,IN,68.00,mist,16.7333,82.2167,
9,Touros,BR,77.00,few clouds,-5.1989,-35.4608,
10,Saint George,US,57.20,clear sky,37.1041,-113.5841,
11,Busselton,AU,62.01,moderate rain,-33.6500,115.3333,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

hotel_not_found_count=0

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.   
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_not_found_count +=1
        print("Hotel not found... skipping.")

print(f'{hotel_not_found_count} hotels not found')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
25 hotels not found


In [18]:
hotel_df.count()

City                   343
Country                343
Max Temp               343
Current Description    343
Lat                    343
Lng                    343
Hotel Name             318
dtype: int64

In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [20]:
# 8a. Create the output File (CSV)
output_data_file= "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Eureka,US,51.01,clear sky,40.8021,-124.1637,Eureka Inn Trademark By Wyndham
1,Bembereke,BJ,71.71,clear sky,10.2283,2.6634,Hotel Bio Guera
3,Cape Town,ZA,70.00,few clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
4,New Norfolk,AU,66.00,scattered clouds,-42.7826,147.0587,The Shingles Riverside Cottages
5,Kapaa,US,73.40,scattered clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
644,Sidlaghatta,IN,55.40,mist,13.3881,77.8628,Murli Dhara
645,Santa Cruz De Tenerife,ES,59.00,overcast clouds,28.4682,-16.2546,Hotel Silken Atlántida Santa Cruz
646,Abu Kamal,SY,51.19,clear sky,34.4506,40.9171,محطة التلفاز
647,Nhamunda,BR,76.51,scattered clouds,-2.1861,-56.7131,Pousada Da Praia


In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and high of {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))